# **Handling Overfitting with CIFAR-10 (Images)**

## **Importing Libraries**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

## **Loading Dataset**

In [2]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [3]:
# Normalize pixel values (0–255 → 0–1)
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [4]:
# Flatten images for Dense NN (instead of CNN)
x_train = x_train.reshape(len(x_train), -1)
x_test = x_test.reshape(len(x_test), -1)

In [5]:
# One-hot encode labels
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

## **Baseline Model (No Regularization)**

In [6]:
def build_baseline():
    model = keras.Sequential([
        layers.Dense(512, activation="relu", input_shape=(x_train.shape[1],)),
        layers.Dense(256, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

baseline_model = build_baseline()
history_baseline = baseline_model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=128,
    verbose=2
)

C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
313/313 - 20s - 63ms/step - accuracy: 0.3006 - loss: 1.9665 - val_accuracy: 0.3717 - val_loss: 1.7842
Epoch 2/50
313/313 - 19s - 60ms/step - accuracy: 0.3789 - loss: 1.7301 - val_accuracy: 0.3986 - val_loss: 1.6949
Epoch 3/50
313/313 - 13s - 42ms/step - accuracy: 0.4129 - loss: 1.6408 - val_accuracy: 0.4070 - val_loss: 1.6614
Epoch 4/50
313/313 - 22s - 69ms/step - accuracy: 0.4393 - loss: 1.5794 - val_accuracy: 0.4399 - val_loss: 1.5889
Epoch 5/50
313/313 - 21s - 66ms/step - accuracy: 0.4561 - loss: 1.5301 - val_accuracy: 0.4573 - val_loss: 1.5447
Epoch 6/50
313/313 - 14s - 45ms/step - accuracy: 0.4703 - loss: 1.4911 - val_accuracy: 0.4618 - val_loss: 1.5232
Epoch 7/50
313/313 - 20s - 65ms/step - accuracy: 0.4798 - loss: 1.4630 - val_accuracy: 0.4559 - val_loss: 1.5216
Epoch 8/50
313/313 - 14s - 45ms/step - accuracy: 0.4890 - loss: 1.4367 - val_accuracy: 0.4534 - val_loss: 1.5292
Epoch 9/50
313/313 - 14s - 46ms/step - accuracy: 0.4990 - loss: 1.4037 - val_accuracy: 0.4797 - 

## **Add Dropout**

In [7]:
def build_dropout():
    model = keras.Sequential([
        layers.Dense(512, activation="relu", input_shape=(x_train.shape[1],)),
        layers.Dropout(0.5),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

dropout_model = build_dropout()
history_dropout = dropout_model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=128,
    verbose=2
)

Epoch 1/50
313/313 - 19s - 61ms/step - accuracy: 0.1831 - loss: 2.1710 - val_accuracy: 0.2783 - val_loss: 2.0087
Epoch 2/50
313/313 - 14s - 45ms/step - accuracy: 0.2182 - loss: 2.0551 - val_accuracy: 0.2877 - val_loss: 1.9717
Epoch 3/50
313/313 - 15s - 49ms/step - accuracy: 0.2379 - loss: 2.0117 - val_accuracy: 0.2917 - val_loss: 1.9643
Epoch 4/50
313/313 - 20s - 63ms/step - accuracy: 0.2481 - loss: 1.9846 - val_accuracy: 0.3138 - val_loss: 1.9426
Epoch 5/50
313/313 - 15s - 46ms/step - accuracy: 0.2533 - loss: 1.9749 - val_accuracy: 0.3121 - val_loss: 1.9659
Epoch 6/50
313/313 - 15s - 48ms/step - accuracy: 0.2592 - loss: 1.9681 - val_accuracy: 0.3026 - val_loss: 1.9753
Epoch 7/50
313/313 - 14s - 45ms/step - accuracy: 0.2619 - loss: 1.9548 - val_accuracy: 0.3215 - val_loss: 1.9396
Epoch 8/50
313/313 - 14s - 45ms/step - accuracy: 0.2688 - loss: 1.9527 - val_accuracy: 0.3344 - val_loss: 1.9322
Epoch 9/50
313/313 - 21s - 68ms/step - accuracy: 0.2737 - loss: 1.9409 - val_accuracy: 0.3157 - 

## **Apply Early Stopping**

In [8]:
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

es_model = build_dropout()
history_es = es_model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=128,
    callbacks=[early_stop],
    verbose=2
)

Epoch 1/50
313/313 - 17s - 54ms/step - accuracy: 0.1882 - loss: 2.1653 - val_accuracy: 0.2907 - val_loss: 1.9871
Epoch 2/50
313/313 - 13s - 41ms/step - accuracy: 0.2285 - loss: 2.0446 - val_accuracy: 0.2848 - val_loss: 1.9528
Epoch 3/50
313/313 - 14s - 45ms/step - accuracy: 0.2431 - loss: 2.0095 - val_accuracy: 0.3098 - val_loss: 1.9445
Epoch 4/50
313/313 - 13s - 42ms/step - accuracy: 0.2493 - loss: 1.9888 - val_accuracy: 0.3281 - val_loss: 1.9411
Epoch 5/50
313/313 - 12s - 40ms/step - accuracy: 0.2526 - loss: 1.9855 - val_accuracy: 0.3095 - val_loss: 1.9619
Epoch 6/50
313/313 - 13s - 42ms/step - accuracy: 0.2673 - loss: 1.9609 - val_accuracy: 0.3230 - val_loss: 1.9390
Epoch 7/50
313/313 - 21s - 66ms/step - accuracy: 0.2641 - loss: 1.9582 - val_accuracy: 0.3105 - val_loss: 1.9607
Epoch 8/50
313/313 - 13s - 40ms/step - accuracy: 0.2712 - loss: 1.9480 - val_accuracy: 0.3387 - val_loss: 1.9370
Epoch 9/50
313/313 - 13s - 40ms/step - accuracy: 0.2740 - loss: 1.9392 - val_accuracy: 0.3363 - 

## **Apply Early Stopping**

In [9]:
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

es_model = build_dropout()
history_es = es_model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=128,
    callbacks=[early_stop],
    verbose=2
)

Epoch 1/50
313/313 - 18s - 56ms/step - accuracy: 0.1895 - loss: 2.1634 - val_accuracy: 0.2808 - val_loss: 2.0100
Epoch 2/50
313/313 - 13s - 41ms/step - accuracy: 0.2145 - loss: 2.0572 - val_accuracy: 0.2655 - val_loss: 2.0125
Epoch 3/50
313/313 - 14s - 46ms/step - accuracy: 0.2302 - loss: 2.0213 - val_accuracy: 0.2976 - val_loss: 1.9618
Epoch 4/50
313/313 - 21s - 66ms/step - accuracy: 0.2447 - loss: 1.9959 - val_accuracy: 0.2931 - val_loss: 1.9647
Epoch 5/50
313/313 - 13s - 40ms/step - accuracy: 0.2468 - loss: 1.9899 - val_accuracy: 0.3033 - val_loss: 1.9585
Epoch 6/50
313/313 - 13s - 41ms/step - accuracy: 0.2513 - loss: 1.9800 - val_accuracy: 0.3044 - val_loss: 1.9534
Epoch 7/50
313/313 - 21s - 67ms/step - accuracy: 0.2548 - loss: 1.9712 - val_accuracy: 0.3125 - val_loss: 1.9450
Epoch 8/50
313/313 - 13s - 40ms/step - accuracy: 0.2533 - loss: 1.9712 - val_accuracy: 0.3093 - val_loss: 1.9549
Epoch 9/50
313/313 - 12s - 40ms/step - accuracy: 0.2589 - loss: 1.9663 - val_accuracy: 0.2959 - 

## **L2 Weight Decay**

In [10]:
def build_l2():
    model = keras.Sequential([
        layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001),
                     input_shape=(x_train.shape[1],)),
        layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
        layers.Dense(10, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

l2_model = build_l2()
history_l2 = l2_model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=128,
    verbose=2
)

Epoch 1/50
313/313 - 19s - 60ms/step - accuracy: 0.3056 - loss: 2.4915 - val_accuracy: 0.3460 - val_loss: 2.1004
Epoch 2/50
313/313 - 16s - 50ms/step - accuracy: 0.3848 - loss: 1.9412 - val_accuracy: 0.3826 - val_loss: 1.8921
Epoch 3/50
313/313 - 15s - 47ms/step - accuracy: 0.4089 - loss: 1.8127 - val_accuracy: 0.4191 - val_loss: 1.7850
Epoch 4/50
313/313 - 14s - 46ms/step - accuracy: 0.4295 - loss: 1.7290 - val_accuracy: 0.4242 - val_loss: 1.7425
Epoch 5/50
313/313 - 15s - 49ms/step - accuracy: 0.4432 - loss: 1.6786 - val_accuracy: 0.4361 - val_loss: 1.7062
Epoch 6/50
313/313 - 15s - 47ms/step - accuracy: 0.4512 - loss: 1.6467 - val_accuracy: 0.4349 - val_loss: 1.7043
Epoch 7/50
313/313 - 16s - 52ms/step - accuracy: 0.4583 - loss: 1.6223 - val_accuracy: 0.4604 - val_loss: 1.6427
Epoch 8/50
313/313 - 19s - 62ms/step - accuracy: 0.4617 - loss: 1.6075 - val_accuracy: 0.4491 - val_loss: 1.6683
Epoch 9/50
313/313 - 16s - 52ms/step - accuracy: 0.4692 - loss: 1.5851 - val_accuracy: 0.4663 - 

## **Depth Experiment**

In [11]:
def build_deeper():
    model = keras.Sequential([
        layers.Dense(1024, activation="relu", input_shape=(x_train.shape[1],)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

deep_model = build_deeper()
history_deep = deep_model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=128,
    verbose=2
)

Epoch 1/50
313/313 - 29s - 93ms/step - accuracy: 0.2944 - loss: 1.9292 - val_accuracy: 0.3589 - val_loss: 1.7875
Epoch 2/50
313/313 - 25s - 79ms/step - accuracy: 0.3797 - loss: 1.7212 - val_accuracy: 0.3840 - val_loss: 1.6932
Epoch 3/50
313/313 - 24s - 76ms/step - accuracy: 0.4106 - loss: 1.6343 - val_accuracy: 0.3974 - val_loss: 1.6819
Epoch 4/50
313/313 - 24s - 76ms/step - accuracy: 0.4370 - loss: 1.5691 - val_accuracy: 0.4078 - val_loss: 1.6444
Epoch 5/50
313/313 - 25s - 81ms/step - accuracy: 0.4571 - loss: 1.5177 - val_accuracy: 0.4367 - val_loss: 1.5866
Epoch 6/50
313/313 - 24s - 78ms/step - accuracy: 0.4712 - loss: 1.4772 - val_accuracy: 0.4562 - val_loss: 1.5350
Epoch 7/50
313/313 - 25s - 80ms/step - accuracy: 0.4861 - loss: 1.4327 - val_accuracy: 0.4757 - val_loss: 1.4828
Epoch 8/50
313/313 - 24s - 76ms/step - accuracy: 0.4986 - loss: 1.4036 - val_accuracy: 0.4662 - val_loss: 1.5036
Epoch 9/50
313/313 - 24s - 77ms/step - accuracy: 0.5033 - loss: 1.3739 - val_accuracy: 0.4803 - 

## **Evaluation & Reporting**

In [12]:
results = {
    "Baseline": baseline_model.evaluate(x_test, y_test, verbose=0),
    "Dropout": dropout_model.evaluate(x_test, y_test, verbose=0),
    "EarlyStopping": es_model.evaluate(x_test, y_test, verbose=0),
    "L2": l2_model.evaluate(x_test, y_test, verbose=0),
    "Deeper": deep_model.evaluate(x_test, y_test, verbose=0),
}

for name, (loss, acc) in results.items():
    print(f"{name} - Test Accuracy: {acc:.4f}")

Baseline - Test Accuracy: 0.5073
Dropout - Test Accuracy: 0.3359
EarlyStopping - Test Accuracy: 0.3166
L2 - Test Accuracy: 0.5053
Deeper - Test Accuracy: 0.4894
